# Hands-On Pertemuan 14: Advanced Machine Learning using Spark MLlib

## Objectives:
- Understand and implement advanced machine learning tasks using Spark MLlib.
- Build and evaluate models using real-world datasets.
- Explore techniques like feature engineering and hyperparameter tuning.


## Introduction to Spark MLlib
Spark MLlib is a scalable library for machine learning that integrates seamlessly with the Spark ecosystem. It supports a wide range of tasks, including regression, classification, clustering, and collaborative filtering.

In [ ]:
!pip install pyspark

In [ ]:
!pip install notebook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.3 MB/s eta 0:00:00


In [ ]:
# Example: Linear Regression with Spark MLlib
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Initialize Spark Session
spark = SparkSession.builder.appName('MLlib Example').getOrCreate()

# Load sample data
data = [(1, 5.0, 20.0), (2, 10.0, 25.0), (3, 15.0, 30.0), (4, 20.0, 35.0)]
columns = ['ID', 'Feature', 'Target']
df = spark.createDataFrame(data, columns)

# Prepare data for modeling
assembler = VectorAssembler(inputCols=['Feature'], outputCol='Features')
df_transformed = assembler.transform(df)

# Train a linear regression model
lr = LinearRegression(featuresCol='Features', labelCol='Target')
model = lr.fit(df_transformed)

# Print model coefficients
print(f'Coefficients: {model.coefficients}')
print(f'Intercept: {model.intercept}')


Coefficients: [0.9999999999999992]
Intercept: 15.000000000000009


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

# Create SparkSession
spark = SparkSession.builder.appName("LogisticRegressionExample").getOrCreate()

# Example dataset
data = [
    (1, Vectors.dense([2.0, 3.0]), 0),
    (2, Vectors.dense([1.0, 5.0]), 1),
    (3, Vectors.dense([2.5, 4.5]), 1),
    (4, Vectors.dense([3.0, 6.0]), 0)
]
columns = ['ID', 'Features', 'Label']

df = spark.createDataFrame(data, columns)

# Train logistic regression model
lr = LogisticRegression(featuresCol='Features', labelCol='Label')
model = lr.fit(df)

# Display coefficients and summary
print(f'Coefficients: {model.coefficients}')
print(f'Intercept: {model.intercept}')


Coefficients: [-12.262057929180484,4.087352266486688]
Intercept: 11.56891272665312


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("KMeansClusteringExample").getOrCreate()

# Example dataset with DenseVector
data = [
    (1, Vectors.dense([1.0, 1.0])),
    (2, Vectors.dense([5.0, 5.0])),
    (3, Vectors.dense([10.0, 10.0])),
    (4, Vectors.dense([15.0, 15.0]))
]
columns = ['ID', 'Features']
df = spark.createDataFrame(data, columns)

# Train KMeans clustering model
kmeans = KMeans(featuresCol='Features', k=2)
model = kmeans.fit(df)

# Show cluster centers
centers = model.clusterCenters()
print(f'Cluster Centers: {centers}')


Cluster Centers: [array([12.5, 12.5]), array([3., 3.])]


## Homework
- Load a real-world dataset into Spark and prepare it for machine learning tasks.
- Build a classification model using Spark MLlib and evaluate its performance.
- Explore hyperparameter tuning using cross-validation.


In [12]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Retail Sales ML") \
    .getOrCreate()

# Load the dataset
data = spark.read.csv("Cleaned_Warehouse_and_Retail_Sales.csv", header=True, inferSchema=True)
data.show(5)

+----+-----+--------------------+---------+--------------------+---------+--------------------+--------------------+-------------------+
|YEAR|MONTH|            SUPPLIER|ITEM CODE|    ITEM DESCRIPTION|ITEM TYPE|        RETAIL SALES|    RETAIL TRANSFERS|    WAREHOUSE SALES|
+----+-----+--------------------+---------+--------------------+---------+--------------------+--------------------+-------------------+
|2020|    1|REPUBLIC NATIONAL...|   100009| BOOTLEG RED - 750ML|     WINE|0.002363876757882...|0.018966944592277216| 0.2987326262587587|
|2020|    1|           PWSWN INC|   100024|MOMENT DE PLAISIR...|     WINE|0.002363876757882...|0.019459720497506553|0.29880920473254974|
|2020|    1|RELIABLE CHURCHIL...|     1001|S SMITH ORGANIC P...|     BEER|0.002363876757882...|0.018966944592277216| 0.2986943370218632|
|2020|    1|LANTERNA DISTRIBU...|   100145|SCHLINK HAUS KABI...|     WINE|0.002363876757882...|0.018966944592277216| 0.2986943370218632|
|2020|    1|DIONYSOS IMPORTS INC|   10029

In [13]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, when, isnull

# Create a Spark session
spark = SparkSession.builder \
    .appName("Retail Sales ML") \
    .getOrCreate()

# Load the dataset
data = spark.read.csv("Cleaned_Warehouse_and_Retail_Sales.csv", header=True, inferSchema=True)
data.show(5)

# Convert categorical columns to numeric using StringIndexer
categorical_columns = ['SUPPLIER', 'ITEM CODE', 'ITEM DESCRIPTION', 'ITEM TYPE']
indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_columns]

# Fit and transform the indexers
for indexer in indexers:
    data = indexer.fit(data).transform(data)

# Convert 'RETAIL SALES' to numeric
data = data.withColumn("RETAIL SALES", col("RETAIL SALES").cast("double"))

# Prepare the feature columns (include indexed columns)
feature_columns = [col + "_index" for col in categorical_columns] + ["RETAIL SALES"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
data = assembler.transform(data)

# Tangani nilai null dan non-string di kolom target
target_column = 'ITEM TYPE'
data = data.withColumn(target_column, \
                       when(isnull(col(target_column)), "").otherwise(col(target_column).cast("string")))

# Gunakan StringIndexer untuk mengubah kolom target menjadi numerik
label_indexer = StringIndexer(inputCol=target_column, outputCol="label")
data = label_indexer.fit(data).transform(data)

# Select only the features and label
data = data.select('features', 'label')

# Split the data into training and test sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Build a classification model using Logistic Regression
lr = LogisticRegression(labelCol='label', featuresCol='features')
model = lr.fit(train_data)

# Evaluate the model
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Hyperparameter tuning using Cross-Validation
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.maxIter, [10, 50, 100]) \
    .build()

crossval = CrossValidator(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           numFolds=3)

cvModel = crossval.fit(train_data)

# Evaluate the best model from cross-validation
cvPredictions = cvModel.transform(test_data)
cvAccuracy = evaluator.evaluate(cvPredictions)
print(f"Cross-Validated Accuracy: {cvAccuracy}")

# Stop the Spark session
spark.stop()

+----+-----+--------------------+---------+--------------------+---------+--------------------+--------------------+-------------------+
|YEAR|MONTH|            SUPPLIER|ITEM CODE|    ITEM DESCRIPTION|ITEM TYPE|        RETAIL SALES|    RETAIL TRANSFERS|    WAREHOUSE SALES|
+----+-----+--------------------+---------+--------------------+---------+--------------------+--------------------+-------------------+
|2020|    1|REPUBLIC NATIONAL...|   100009| BOOTLEG RED - 750ML|     WINE|0.002363876757882...|0.018966944592277216| 0.2987326262587587|
|2020|    1|           PWSWN INC|   100024|MOMENT DE PLAISIR...|     WINE|0.002363876757882...|0.019459720497506553|0.29880920473254974|
|2020|    1|RELIABLE CHURCHIL...|     1001|S SMITH ORGANIC P...|     BEER|0.002363876757882...|0.018966944592277216| 0.2986943370218632|
|2020|    1|LANTERNA DISTRIBU...|   100145|SCHLINK HAUS KABI...|     WINE|0.002363876757882...|0.018966944592277216| 0.2986943370218632|
|2020|    1|DIONYSOS IMPORTS INC|   10029

Py4JJavaError: An error occurred while calling o2182.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 164.0 failed 1 times, most recent failure: Lost task 0.0 in stage 164.0 (TID 160) (e653b5336dc1 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda$3673/0x000000084156d040`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIt